# Origens e Destinos de Ônibus Rodoviários no Brasil

In [2]:
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import matplotlib
from matplotlib.cm import Set2
from tqdm.auto import tqdm

coords_path = '../data/2022-12-17 15:52:24.006277-coords.json'
data_path = '../data/2022-12-17 14:46:49.632892-output.json'
routes_path = '../data/2022-12-18 01:33:21.284286-company-routes.json'
SLUG_PATH = '../data/slug_df.csv.xz'

mapbox_access_token = open("../config/.mapbox_token").read()
px.set_mapbox_access_token(mapbox_access_token)

In [3]:
slug_df = pd.read_csv(SLUG_PATH)
categories = set(slug_df.company)
n_colors = len(categories)
colors = px.colors.sample_colorscale("turbo", [n/(n_colors -1) for n in range(n_colors)])
colors = {category: colors[i] for i, category in enumerate(categories)}
slug_df = slug_df.assign(company_color=lambda df: df.company.map(colors))

In [4]:
slug = 'sao-paulo-sp'
src_as_reference = True

def get_slug_fig_df(slug_df, slug, src_as_reference):
    if src_as_reference:
        city_query = f"src == '{slug}'"
        reference_col = 'dst'
        adjacent_col = 'src'
    else:
        city_query = f"dst == '{slug}'"
        reference_col = 'src'
        adjacent_col = 'dst'
    QUERY = f"{city_query} & align_to_src == {src_as_reference}"
    fig_df = slug_df.query(QUERY).reset_index()
    return reference_col, adjacent_col, fig_df

reference_col, adjacent_col, fig_df = get_slug_fig_df(slug_df, slug, src_as_reference)


# #inds = slug_df.company.isin(['vallesul'])
# fig_df = slug_df
#fig_df = fig_df[fig_df.dst.map(lambda x: "-ms" in x or "-go" in x or "-pr" in x or "-mt" in x if type(x) is str else False)]

fig = px.scatter_mapbox(fig_df,
                     lon=f'{reference_col}_x',
                     lat=f'{reference_col}_y', 
                     color='company',
                     hover_name=reference_col,
                     height=1000, width=1200)

fig.update_traces(marker=dict(size=8))

# fig.add_trace(go.Scattermapbox(
#     name='source',
#     lon = [fig_df[f'{adjacent_col}_x'].unique()[0]],
#     lat = [fig_df[f'{adjacent_col}_y'].unique()[0]],
#     mode = 'markers',
#     marker = dict(
#         symbol='square',
#         size = 8,
#         color = 'white'
#     )
#     )
#     )

# fig.update_geos(fitbounds="locations", projection_type='orthographic',
#     resolution=50,
#     showcoastlines=True, coastlinecolor="RebeccaPurple",
#     showocean=True, oceancolor="LightBlue",
#     showlakes=True, lakecolor="LightBlue",
#     showrivers=True, rivercolor="LightBlue",
#         showcountries=True, countrycolor="Black")
fig.update_layout(
    mapbox = {
        'accesstoken': mapbox_access_token,
        'style': "outdoors", 'zoom': 3},
    showlegend = True)

fig.show()

In [5]:
slugs_list = slug_df.reset_index().src.value_counts().index[0:3]

fig = go.Figure()

btn_bool_list = [False] * len(slugs_list)
buttons = []
for i, slug in enumerate(slugs_list):
    reference_col, adjacent_col, fig_df = get_slug_fig_df(slug_df, slug, True)
    fig.add_trace(go.Scattergeo(
        name=slug,
        lon=fig_df[f'{reference_col}_x'],
        lat=fig_df[f'{reference_col}_y'],
        mode='markers',
        text=fig_df[f'{reference_col}'],
        marker=dict(color=fig_df['company_color'])
    ))

    bool_list = btn_bool_list.copy()
    bool_list[i] = True
    button_args = [{'visible': bool_list}, {'title': slug}]
    button = dict(label=slug, 
                  method='update', 
                  args=button_args)
    buttons.append(button)


fig.update_layout(updatemenus=[dict(active=0,
                                     buttons=buttons)])

fig.update_geos(fitbounds="locations",
    resolution=110,
    showcoastlines=True, coastlinecolor="RebeccaPurple",
    showocean=True, oceancolor="LightBlue",
    showlakes=True, lakecolor="LightBlue",
    showrivers=True, rivercolor="LightBlue",
        showcountries=True, countrycolor="Black")
fig.show()

In [6]:


fig_df = slug_df.copy()

top = fig_df.src.value_counts().sort_values(ascending=False).index[0:5]

fig_df = fig_df[fig_df.src.isin(top)]

fig = px.scatter_mapbox(fig_df,
                     lon=f'dst_x',
                     lat=f'dst_y', 
                     color='company',
                     animation_frame='src',
                     #animation_group='src',
                     hover_name='dst',
                     height=800, width=1500)

fig.update_traces(marker=dict(size=5))

# fig.add_trace(go.Scattermapbox(
#     name='source',
#     lon = [fig_df[f'{adjacent_col}_x'].unique()[0]],
#     lat = [fig_df[f'{adjacent_col}_y'].unique()[0]],
#     mode = 'markers',
#     marker = dict(
#         symbol='square',
#         size = 8,
#         color = 'white'
#     )
#     )
#     )

# fig.update_geos(fitbounds="locations", projection_type='orthographic',
#     resolution=50,
#     showcoastlines=True, coastlinecolor="RebeccaPurple",
#     showocean=True, oceancolor="LightBlue",
#     showlakes=True, lakecolor="LightBlue",
#     showrivers=True, rivercolor="LightBlue",
#         showcountries=True, countrycolor="Black")
fig.update_layout(
    mapbox = {
        'accesstoken': mapbox_access_token,
        'style': "dark", 'zoom': 3},
    showlegend = True)

fig.show()

In [9]:
slug_df.src.value_counts()

sao-paulo-sp               4387
goiania-go                 2503
brasilia-df                1904
rio-de-janeiro-rj          1691
campo-grande-ms            1590
                           ... 
nova-bandeirantes-mt          2
imaculada-pb                  2
veredinha-mg                  2
guaranta-sp                   2
nova-iguacu-de-goias-go       2
Name: src, Length: 2224, dtype: int64